# Fixing Flags
Embedded in MET_DataProcess2FACE

In [1]:
import pandas as pd

In [102]:
# paths

paths = {}
paths ["ELM-DUKE"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/ELM_Data/data/atm/datm7/CLM1PT_data/1x1pt_US-DUK/"
paths ["FACEMDS_Walker2018"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Walker_2018_FATES_MDS/data/"
paths ["DukeFACE_Oren2022"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_DUKE_Met/data/"
paths ["Raleigh_Airport_Met"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/"
paths ["ERA5_Met"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/ERA5_Duke_Met/"
paths ["NARR_Met"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/NARR_Met/"
paths ["AmeriFlux"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/AmeriFlux/"
paths ["Save_Processed"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/"


In [103]:
df = pd.read_csv('/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_h.csv')
df_fv = pd.read_csv('/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_h_fv.csv')

/Users/ud4/opt/anaconda3/envs/pyces/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (5,7,9,11,13,19,21,23,25,27,29,31,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [104]:
df_fv.head()

,Unnamed: 0,YEAR,DTIME,DOY,HRMIN,Rainf,Rainf_f,Tair,Tair_f,RH,...,PSurf,PSurf_f,aCO2,eCO2,Ndep,SolarElevation,aCO2_f,eCO2_f,Ndep_f,SolarElevation_f
0,0,NaN,NaN,NaN,NaN,kg/m2/s,NaN,K,NaN,%,...,Pa,NaN,ppmv,ppmv,g/m2/(30-minute),degree,NaN,NaN,NaN,NaN
1,1,1996.0,1.020833,1.0,30.0,2.8e-05,0.0,282.05,0.0,100.0,...,99197.625,0.0,374.7482,374.7482,7.79827e-05,-76.97134,0.0,0.0,0.0,0.0
2,2,1996.0,1.069444,1.0,100.0,2.8e-05,0.0,281.75,0.0,100.0,...,99197.625,0.0,374.7482,374.7482,7.79827e-05,-75.95279,0.0,0.0,0.0,0.0
3,3,1996.0,1.090278,1.0,130.0,2.8e-05,0.0,281.45,0.0,100.0,...,99229.12,0.0,374.7482,374.7482,7.79827e-05,-72.4138,0.0,0.0,0.0,0.0
4,4,1996.0,1.138889,1.0,200.0,2.8e-05,0.0,281.45,0.0,100.0,...,99229.12,0.0,374.7482,374.7482,7.79827e-05,-67.52518,0.0,0.0,0.0,0.0


In [105]:
df_fv.columns

Index(['Unnamed: 0', 'YEAR', 'DTIME', 'DOY', 'HRMIN', 'Rainf', 'Rainf_f',
       'Tair', 'Tair_f', 'RH', 'RH_f', 'VPD', 'VPD_f', 'PAR', 'PAR_f', 'SM',
       'SM_f', 'SWP', 'SWP_f', 'SVP', 'SVP_f', 'Rn', 'Rn_f', 'SLT', 'SLT_f',
       'SWdown', 'SWdown_f', 'LWdown', 'LWdown_f', 'Wind', 'Wind_f', 'PSurf',
       'PSurf_f', 'aCO2', 'eCO2', 'Ndep', 'SolarElevation', 'aCO2_f', 'eCO2_f',
       'Ndep_f', 'SolarElevation_f'],
      dtype='object')

In [106]:
import pickle
fname_start_dates = f"{paths['Save_Processed']}start_dates_Oren.pkl"
# Open the pickle file for reading in binary mode
with open(fname_start_dates, 'rb') as file:
    dict_start_dates = pickle.load(file)

# Copied from FACE MDS existing till the start date of dataset
for col_name in ['PAR','RH','Tair','Rainf','VPD'] :
    flag_col = f'{col_name}_f'
    year_tmp = dict_start_dates [col_name].Year
    doy_tmp = dict_start_dates [col_name].DOY
    time_tmp = dict_start_dates [col_name].Time
    filter_tmp = df_fv[(df_fv['YEAR'] == year_tmp) & (df_fv['DOY'] == doy_tmp) & (df_fv['HRMIN'] == time_tmp)]
    df_fv[flag_col].iloc[1:filter_tmp.index[0]] = 2

# ERA5 Source
for col_name in ['Wind','PSurf']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 4
    
# Copied from FACE MDS existing
for col_name in ['aCO2', 'eCO2', 'Ndep', 'SolarElevation']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 2
    
# Derived based on a formula and other variabls
for col_name in ['SWdown', 'LWdown']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 1

{'Tair': Year    1997
 DOY      125
 Time    1730
 Tair    21.5
 Name: 0, dtype: object,
 'Rainf': Year     1997
 DOY       126
 Time       30
 Rainf     0.0
 Name: 0, dtype: object,
 'RH': Year    1997
 DOY      125
 Time    1730
 RH      0.43
 Name: 0, dtype: object,
 'SM': Year    1997
 DOY      101
 Time     100
 SM      0.28
 Name: 0, dtype: object,
 'SWP': Year    2007.00
 DOY        1.00
 Time      30.00
 SWP      478.84
 Name: 0, dtype: float64,
 'SVP': Year    1997
 DOY      125
 Time    1730
 SVP     2.56
 Name: 0, dtype: object,
 'VPD': Year    1997
 DOY      125
 Time    1730
 VPD     1.47
 Name: 0, dtype: object,
 'SLT': Year     1997
 DOY       143
 Time       30
 SLT     13.92
 Name: 0, dtype: object,
 'PAR': Year    1997
 DOY        1
 Time      30
 PAR      0.0
 Name: 0, dtype: object,
 'Rn': Year      1997
 DOY          1
 Time        30
 Rn     -6999.0
 Name: 0, dtype: object}

In [107]:
dict_start_dates.keys()

dict_keys(['Tair', 'Rainf', 'RH', 'SM', 'SWP', 'SVP', 'VPD', 'SLT', 'PAR', 'Rn'])

In [115]:
for col_name in ['PAR','RH','Tair','Rainf','VPD'] :
    flag_col = f'{col_name}_f'
    year_tmp = dict_start_dates [col_name].Year
    doy_tmp = dict_start_dates [col_name].DOY
    time_tmp = dict_start_dates [col_name].Time
    filter_tmp = df_fv[(df_fv['YEAR'] == year_tmp) & (df_fv['DOY'] == doy_tmp) & (df_fv['HRMIN'] == time_tmp)]
    df_fv[flag_col].iloc[1:filter_tmp.index[0]] = 2

/Users/ud4/opt/anaconda3/envs/pyces/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [116]:
# ERA5 Source
for col_name in ['Wind','PSurf']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 4

/var/folders/f1/01gxw8vn74q_x_rf_p5ztryjr405zq/T/ipykernel_45405/1802289720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fv[flag_col][1:] = 4


In [117]:
# Copied from FACE MDS existing
for col_name in ['aCO2', 'eCO2', 'Ndep', 'SolarElevation']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 2

/var/folders/f1/01gxw8vn74q_x_rf_p5ztryjr405zq/T/ipykernel_45405/806918473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fv[flag_col][1:] = 2


In [118]:
# Derived based on a formula and other variabls
for col_name in ['SWdown', 'LWdown']:
    flag_col = f'{col_name}_f'
    df_fv[flag_col][1:] = 1

/var/folders/f1/01gxw8vn74q_x_rf_p5ztryjr405zq/T/ipykernel_45405/1606850210.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fv[flag_col][1:] = 1


In [119]:
df_fv.iloc[:,:20].head()

,Unnamed: 0,YEAR,DTIME,DOY,HRMIN,Rainf,Rainf_f,Tair,Tair_f,RH,RH_f,VPD,VPD_f,PAR,PAR_f,SM,SM_f,SWP,SWP_f,SVP
0,0,NaN,NaN,NaN,NaN,kg/m2/s,NaN,K,NaN,%,NaN,Pa,NaN,umol/m2/s,NaN,NaN,NaN,NaN,NaN,kPa
1,1,1996.0,1.020833,1.0,30.0,2.8e-05,2.0,282.05,2.0,100.0,2.0,0.0,2.0,0.0,2.0,-6999.0,0.0,-6999.0,0.0,-6999.0
2,2,1996.0,1.069444,1.0,100.0,2.8e-05,2.0,281.75,2.0,100.0,2.0,0.0,2.0,0.0,2.0,-6999.0,0.0,-6999.0,0.0,-6999.0
3,3,1996.0,1.090278,1.0,130.0,2.8e-05,2.0,281.45,2.0,100.0,2.0,0.0,2.0,0.0,2.0,-6999.0,0.0,-6999.0,0.0,-6999.0
4,4,1996.0,1.138889,1.0,200.0,2.8e-05,2.0,281.45,2.0,100.0,2.0,0.0,2.0,0.0,2.0,-6999.0,0.0,-6999.0,0.0,-6999.0


In [120]:
df_fv.iloc[:,20:30].head()

,SVP_f,Rn,Rn_f,SLT,SLT_f,SWdown,SWdown_f,LWdown,LWdown_f,Wind
0,NaN,umol/m2/s,NaN,K,NaN,W/m2,NaN,W/m2,NaN,m/s
1,0.0,-6999.0,0.0,-6999.0,0.0,0.0,1.0,297.2362,1.0,1.0033706
2,0.0,-6999.0,0.0,-6999.0,0.0,0.0,1.0,295.162,1.0,1.0033706
3,0.0,-6999.0,0.0,-6999.0,0.0,0.0,1.0,293.0988,1.0,1.4305817
4,0.0,-6999.0,0.0,-6999.0,0.0,0.0,1.0,293.0988,1.0,1.4305817


In [121]:
df_fv.iloc[:,30:].head()

,Wind_f,PSurf,PSurf_f,aCO2,eCO2,Ndep,SolarElevation,aCO2_f,eCO2_f,Ndep_f,SolarElevation_f
0,NaN,Pa,NaN,ppmv,ppmv,g/m2/(30-minute),degree,NaN,NaN,NaN,NaN
1,4.0,99197.625,4.0,374.7482,374.7482,7.79827e-05,-76.97134,2.0,2.0,2.0,2.0
2,4.0,99197.625,4.0,374.7482,374.7482,7.79827e-05,-75.95279,2.0,2.0,2.0,2.0
3,4.0,99229.12,4.0,374.7482,374.7482,7.79827e-05,-72.4138,2.0,2.0,2.0,2.0
4,4.0,99229.12,4.0,374.7482,374.7482,7.79827e-05,-67.52518,2.0,2.0,2.0,2.0


In [114]:
flag_col

'LWdown_f'

In [101]:
df_fv[flag_col]

0         NaN
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
298075    1.0
298076    1.0
298077    1.0
298078    1.0
298079    1.0
Name: LWdown_f, Length: 298080, dtype: float64